In [1]:
import json



In [3]:
# Input and output file names
input_file = "./business_qlora.jsonl"
output_file = "./my_dataset.json"

data = []

# Read JSONL file and convert it to the required format
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line.strip())
        formatted_entry = {
            "conversations": [
                {"from": "human", "value": entry["instruction"]},
                {"from": "gpt", "value": entry["response"]}
            ]
        }
        data.append(formatted_entry)

# Write the formatted data to a JSON file
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

print(f"Converted JSONL to JSON and saved to {output_file}")


Converted JSONL to JSON and saved to ./my_dataset.json
